In [ ]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
df = pd.read_csv("/content/blogs (1).csv")

In [ ]:
df.head()

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism


In [ ]:
# Check for missing values
df.isnull().sum()

,0
Data,0
Labels,0


In [ ]:
# Value counts of labels
df['Labels'].value_counts()

,count
Labels,
alt.atheism,100
comp.graphics,100
talk.politics.misc,100
talk.politics.mideast,100
talk.politics.guns,100
soc.religion.christian,100
sci.space,100
sci.med,100
sci.electronics,100


In [ ]:
# Download stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [ ]:
df['cleaned_text'] = df['Data'].apply(preprocess_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['cleaned_text']).toarray()
y = df['Labels']

 Naive Bayes Model for Text Classification

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

In [ ]:
# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
classification_report(y_test, y_pred)

Accuracy: 0.725


'                          precision    recall  f1-score   support\n\n             alt.atheism       0.47      0.78      0.58        18\n           comp.graphics       0.52      0.72      0.60        18\n comp.os.ms-windows.misc       0.77      0.77      0.77        22\ncomp.sys.ibm.pc.hardware       0.79      0.60      0.68        25\n   comp.sys.mac.hardware       0.67      0.67      0.67        21\n          comp.windows.x       0.83      0.80      0.82        25\n            misc.forsale       0.75      0.67      0.71        18\n               rec.autos       0.75      1.00      0.86        18\n         rec.motorcycles       0.57      0.81      0.67        16\n      rec.sport.baseball       0.85      0.94      0.89        18\n        rec.sport.hockey       0.83      1.00      0.91        15\n               sci.crypt       0.85      0.89      0.87        19\n         sci.electronics       0.53      0.50      0.52        16\n                 sci.med       0.81      0.76      0.79    

Sentiment Analysis

In [ ]:
from textblob import TextBlob

In [ ]:
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [ ]:
df['sentiment'] = df['Data'].apply(get_sentiment)

In [ ]:
sentiment_distribution = df.groupby('Labels')['sentiment'].value_counts(normalize=True).unstack()
sentiment_distribution

sentiment,negative,positive
Labels,,
alt.atheism,0.23,0.77
comp.graphics,0.24,0.76
comp.os.ms-windows.misc,0.22,0.78
comp.sys.ibm.pc.hardware,0.20,0.80
comp.sys.mac.hardware,0.24,0.76
comp.windows.x,0.27,0.73
misc.forsale,0.16,0.84
rec.autos,0.17,0.83
rec.motorcycles,0.26,0.74


Evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [ ]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.7358169248649117
Recall: 0.725
F1 Score: 0.7157259681427744
